# 16.05 Classwork №1: parse news site to create corpus

In [25]:
import lxml.html as html
from pandas import DataFrame
from bs4 import BeautifulSoup
import urllib3
import json
import requests

Будем парсить The Guardian. Они предоставляют бесплатный API для разработчиков на следующих условиях: 12 вызовов в секунду, 5 000 за день, доступ к полному тексту статьи открыт. Возвращаемый формат - JSON.
Для получения ключа нужно пройти регистрацию по ссылке: http://open-platform.theguardian.com/access/ . В ответ на почту сразу высылается api-key.

Информация о корпусе:
<br> - Время: с 1 января 2016 до 28 мая 2016
<br> - Категории: 5 (новости Великобритании, мировые новости, спорт, культура, мода)
<br> - Выдача на 1 категорию: 200 статей
<br> - Поля: ссылка, заголовок, полный текст, категория, тэги.
При желании можно так же вытащить дату публикации на сайте и в печати, информацию об авторе, тип (статья или блог), изменялся ли текст, если к статье есть фото, то имя фотографа. 

In [91]:
site_url = 'http://content.guardianapis.com/search?section='
api_key = '&from-date=2016-01-01&page-size=200&show-tags=all&show-blocks=all&api-key=cc63bd2e-e220-459f-82c1-2e9943816bf9'
categories = ['uk-news', 'world', 'sport', 'culture', 'fashion']

corpus = {} #{section: category, articles: {url: url, attrs: {text: text, title :title,  tags: [list_of_tags]} } }
for category in categories: 
    r = requests.get(site_url+category+api_key)
    content_string = r.content.decode("utf-8")
    content_json = json.loads(content_string)
    attrs = {}
    articles = {}
    for item in content_json['response']['results']:
        attrs['title'] = item['webTitle']
        attrs['section'] = item['sectionId']
        tags = []
        for i in range(len( item['tags'])):
            #print ('tags: ', item['tags'][i]['id'])
            tags.append(item['tags'][i]['id'])
        #print (item['sectionId'], ': ', item['webTitle'], item['webUrl'])    
        #print (item['blocks']['body'][0]['bodyTextSummary'])
        attrs['tags'] = tags
        attrs['text'] = item['blocks']['body'][0]['bodyTextSummary']     
        articles[item['webUrl']] = attrs  
    corpus[category] = articles   

In [93]:
for k,v in corpus.items():
    print (k, len(v.keys()))

world 200
uk-news 200
sport 200
culture 200
fashion 200
